In [1]:
import pulp

In [13]:

soc = [80,20,60]
n = len(soc)
stations = [20,40,100]
charge_points_list = [1,1]
#stations = [90,120,150]

speed = 10
charging_rate_station = 5
ex_charging_rate_bess = 10
arrival_time = [0,1,3]

T = 1000
bess_capacity = 300
bess_init = [250,250]
kms_to_soc = 1

for i in range(len(stations)):
    stations[i]=stations[i]*kms_to_soc
speed = speed*kms_to_soc
print(speed)
print(stations)

time_discre = 1
for i in range(len(arrival_time)):
    arrival_time[i] *= time_discre
speed = speed/time_discre
charging_rate_station = charging_rate_station/time_discre
ex_charging_rate_bess = ex_charging_rate_bess/time_discre



10
[20, 40, 100]


In [14]:
#initiate the model
cs = pulp.LpProblem("station",pulp.LpMinimize)
vehicles=[]
s_names = []
#add variables
soc_vehicle = {}
arrival_time_vehicle = {}
charge_points = {}
for i in range(n):
    vehicles.append(f"V{i}")
    soc_vehicle[f"V{i}"] = soc[i]
    arrival_time_vehicle[f"V{i}"] = arrival_time[i]
m = len(stations)-1
bess_init_name = {}
for i in range(m):
    s_names.append(f"station{i+1}")
    bess_init_name[f"station{i+1}"] = bess_init[i]
    charge_points[f"station{i+1}"] = charge_points_list[i]
print(charge_points)
print(s_names)
print(bess_init_name)
M = 100000
print(vehicles)

#defining 
a = {}
s = {}
u = {}
d = {}
o = {}
sb = {}
b = {}
bc = {}
charge = {}
bac = {}

for station in s_names:
    #arrival time at stations
    a[station] = pulp.LpVariable.dict(f"Arrival Time {station}",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
    
    # servicing time at stations
    s[station] = pulp.LpVariable.dict(f"Service Time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    #initial charging time at sations
    u[station] = pulp.LpVariable.dict(f"initial charge time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    #final charge time at stations
    d[station] = pulp.LpVariable.dict(f"final charge time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    # sigma- temporal ordering of vehicles
    o[station] = pulp.LpVariable.dict(f"sigma {station}",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
    
    # to see if vehicle is gonna use the respective station
    sb[station] = pulp.LpVariable.dict(f"Service Time {station} binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
    
    #bess servicing times at stations
    b[station] = pulp.LpVariable.dict(f"BESS service Time {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    # charge in bess available for vehicles
    bc[station] = pulp.LpVariable.dict(f"BESS charge {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
    
    # if bess reached max charge or not
    charge[station] = pulp.LpVariable.dict(f"reached max charge {station}",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpBinary)
    
    #amount to charge
    bac[station] = pulp.LpVariable.dict(f"BESS charge for vehicle {station}",[vehicle for vehicle in vehicles],cat=pulp.LpInteger)

#finding arrival times for each station.
for i in range(len(s_names)):
    for vehicle in vehicles:
        if i == 0:
            cs += a[s_names[i]][vehicle] == arrival_time_vehicle[vehicle]+(stations[0]/speed)
        else:
            cs += a[s_names[i]][vehicle] == (stations[i]/speed) + d[s_names[i-1]][vehicle]

#ensuring the vehicle leaves by the end of the end of the max Time
for vehicle in vehicles:
    cs += u[s_names[-1]][vehicle] <= T-s[s_names[-1]][vehicle]

#finding distance travelled by each truck and ensuring it it possible
dist = 0
for i in range(len(s_names)):
    dist += stations[i]
    for vehicle in vehicles:
        cs += charging_rate_station*(pulp.lpSum(s[s_names[j]][vehicle] for j in range(i+1)))+ex_charging_rate_bess*(pulp.lpSum(b[s_names[j]][vehicle] for j in range(i+1))) + soc_vehicle[vehicle] >= dist + stations[i+1]
        cs += charging_rate_station*(pulp.lpSum(s[s_names[j]][vehicle] for j in range(i+1)))+ex_charging_rate_bess*(pulp.lpSum(b[s_names[j]][vehicle] for j in range(i+1))) + soc_vehicle[vehicle] -dist <= 100

#constraints
for station in s_names:
    for vehicle in vehicles:
        
        #departure = service + charge start
        cs += d[station][vehicle] == s[station][vehicle]+u[station][vehicle]

        # charge start > arival time
        cs += u[station][vehicle]>=a[station][vehicle]

        #to see if a vehicle wants to cross the station or charge there
        cs += M*sb[station][vehicle] >=s[station][vehicle]
        cs += sb[station][vehicle] <= s[station][vehicle]
        
        # bess time should be less than total charging time at 
        cs += b[station][vehicle]<=s[station][vehicle]
        cs += b[station][vehicle]*ex_charging_rate_bess<=bc[station][vehicle]
        
        #checking if the amount we charge is greater than 100 or not
        cs += M*charge[station][vehicle]>=bess_init_name[station]+bac[station][vehicle]-bess_capacity
        cs += M*(1-charge[station][vehicle]) >= -(bess_init_name[station]+bac[station][vehicle]-bess_capacity)

        # implementing max of bess capaicty
        cs += bc[station][vehicle]>= bess_capacity - M*(1-charge[station][vehicle])
        cs += bc[station][vehicle]>= bess_init_name[station] + bac[station][vehicle] - M*(charge[station][vehicle])
        cs += bc[station][vehicle]<= bess_capacity + M*(1-charge[station][vehicle])
        cs += bc[station][vehicle]<= bess_init_name[station] + bac[station][vehicle] + M*(charge[station][vehicle])  
            
    # for bess implementations
    for station in s_names:
        for i in range(len(vehicles)):
            #calculating how much charge is available for a vehicle in the bess
            cs += bac[station][vehicles[i]] ==  (u[station][vehicles[i]]-u[station][vehicles[0]] - 2*pulp.lpSum([b[station][vehicles[j]]] for j in range(i-1,-1,-1)))*ex_charging_rate_bess    # for waiting time implementation 
        for vehicle1 in vehicles:
            for vehicle2 in vehicles:
                if vehicle1!=vehicle2:
                    
                    #making sure o1[vehicle1+vehicle2] = 1 when vehicle1 in front of vehicle2
                    cs += M*o[station][vehicle1+vehicle2] >= a[station][vehicle2]-a[station][vehicle1]
                    cs += M*(1-o[station][vehicle1+vehicle2])>= a[station][vehicle1]-a[station][vehicle2]
                    
                    #implementing wait time
                    cs += u[station][vehicle2]-u[station][vehicle1]-s[station][vehicle1]>=(o[station][vehicle1+vehicle2]-1)*T-M*(1-sb[station][vehicle2])                
                    
                    #making sure only one vehicle is front of another
                    cs += o[station][vehicle1+vehicle2] + o[station][vehicle2+vehicle1] >= 1


{'station1': 1, 'station2': 1}
['station1', 'station2']
{'station1': 250, 'station2': 250}
['V0', 'V1', 'V2']


In [15]:
cs

station:
MINIMIZE
None
SUBJECT TO
_C1: Arrival_Time_station1_V0 = 2

_C2: Arrival_Time_station1_V1 = 3

_C3: Arrival_Time_station1_V2 = 5

_C4: Arrival_Time_station2_V0 - final_charge_time_station1_V0 = 4

_C5: Arrival_Time_station2_V1 - final_charge_time_station1_V1 = 4

_C6: Arrival_Time_station2_V2 - final_charge_time_station1_V2 = 4

_C7: Service_Time_station2_V0 + initial_charge_time_station2_V0 <= 1000

_C8: Service_Time_station2_V1 + initial_charge_time_station2_V1 <= 1000

_C9: Service_Time_station2_V2 + initial_charge_time_station2_V2 <= 1000

_C10: 10 BESS_service_Time_station1_V0 + 5 Service_Time_station1_V0 >= -20

_C11: 10 BESS_service_Time_station1_V0 + 5 Service_Time_station1_V0 <= 40

_C12: 10 BESS_service_Time_station1_V1 + 5 Service_Time_station1_V1 >= 40

_C13: 10 BESS_service_Time_station1_V1 + 5 Service_Time_station1_V1 <= 100

_C14: 10 BESS_service_Time_station1_V2 + 5 Service_Time_station1_V2 >= 0

_C15: 10 BESS_service_Time_station1_V2 + 5 Service_Time_station1_

In [5]:
cs += pulp.lpSum([d[station][vehicle]-a[station][vehicle]] for station in s_names for vehicle in vehicles)
cs.solve()

pulp.LpStatus[cs.status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sushi/Desktop/internship/Routing/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/7cb9ddd68c5f47a9a6fe8b197aa5a05a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/7cb9ddd68c5f47a9a6fe8b197aa5a05a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 42 RHS
At line 43 BOUNDS
At line 56 ENDATA
Problem MODEL has 0 rows, 12 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -6000 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -6000 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 

'Optimal'

In [6]:
for station in s_names:
    for vehicle in vehicles:
        print(f"{vehicle} waits at {station} for {u[station][vehicle].varValue-a[station][vehicle].varValue}")
        print(f"{vehicle} charges at {station} for {b[station][vehicle].varValue} using the bess and station, and for {s[station][vehicle].varValue-b[station][vehicle].varValue} only using station")
        print(f"{vehicle} reaches {station} at {a[station][vehicle].varValue}")
        print(f"bess energy level for {vehicle} is {bc[station][vehicle].varValue} at {station}")
        print(f"{vehicle} charges {station} at {u[station][vehicle].varValue}")
        print(f"{vehicle} charge at {station} at {charge[station][vehicle].varValue, charge[station][vehicle].varValue}")

        print()


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [7]:


#objective function
print("wait and charging time is ",pulp.value(cs.objective))

state = {}
for vehicle in vehicles:
    state[vehicle] = []
max_len = 0
for i in range(len(s_names)):
    for vehicle in vehicles:
        for l in range(int(stations[i]/speed)):
            state[vehicle].append("driving")
        for j in range(int(u[s_names[i]][vehicle].varValue-a[s_names[i]][vehicle].varValue)):
            state[vehicle].append("waiting")
        for k in range(int(s[s_names[i]][vehicle].varValue)):
            state[vehicle].append("charging")
for vehicle in vehicles:
    for l in range(int(stations[-1]/speed)):
        state[vehicle].append("driving")
    if len(state[vehicle])>max_len:
        max_len = len(state[vehicle])
for vehicle in vehicles:
    if len(state[vehicle]) < max_len:
        for l in range(max_len-len(state[vehicle])):
            state[vehicle].append("reached")


# state is first graph variable     max_ len is max time       
            
soc_trucks = {}
max_len_soc = 0
for vehicle in vehicles:
    soc_trucks[vehicle] = [soc_vehicle[vehicle]]
for i in range(len(s_names)):
    for vehicle in vehicles:
        for j in range(int(stations[i]/speed)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
        for j in range(int(u[s_names[i]][vehicle].varValue-a[s_names[i]][vehicle].varValue)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
        for k in range(int(b[s_names[i]][vehicle].varValue)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+ex_charging_rate_bess+charging_rate_station)
        for j in range(int(s[s_names[i]][vehicle].varValue-b[s_names[i]][vehicle].varValue)):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1]+charging_rate_station)
for vehicle in vehicles:
    for j in range(int(stations[-1]/speed)):
        soc_trucks[vehicle].append(soc_trucks[vehicle][-1]-speed)
    if len(soc_trucks[vehicle])>max_len_soc:
        max_len_soc = len(soc_trucks[vehicle])
for vehicle in vehicles:
    if len(soc_trucks[vehicle]) < max_len_soc:
        for j in range(max_len_soc-len(soc_trucks[vehicle])):
            soc_trucks[vehicle].append(soc_trucks[vehicle][-1])
soc_bess = []
bess_state = []
for i in range(len(s_names)):
    soc_bess.append([bess_init[i]])
    bess_state.append([])
for i  in range(len(s_names)):
    for j  in range(len(vehicles)):
        if j == 0:
            for k in range(int(u[s_names[i]][vehicles[j]].varValue)):
                bess_state[i].append("idle")
                soc_bess[i].append(soc_bess[i][-1])
            for k in range(int(b[s_names[i]][vehicles[j]].varValue)):
                bess_state[i].append("discharging")
                soc_bess[i].append(soc_bess[i][-1]-ex_charging_rate_bess)
            for k in range(int(s[s_names[i]][vehicles[j]].varValue-b[s_names[i]][vehicles[j]].varValue)):
                if soc_bess[i][-1] >= bess_capacity:
                    bess_state[i].append("idle")
                    soc_bess[i].append(soc_bess[i][-1])
                else:
                    bess_state[i].append("charging")
                    soc_bess[i].append(min(bess_capacity,soc_bess[i][-1]+ex_charging_rate_bess))
        else:
            for k in range(int(u[s_names[i]][vehicles[j]].varValue-u[s_names[i]][vehicles[j-1]].varValue-b[s_names[i]][vehicles[j-1]].varValue)):
                if soc_bess[i][-1] >= bess_capacity:
                    bess_state[i].append("idle")
                    soc_bess[i].append(soc_bess[i][-1])
                else:
                    bess_state[i].append("charging")
                    soc_bess[i].append(min(bess_capacity,soc_bess[i][-1]+ex_charging_rate_bess))
            for k in range(int(b[s_names[i]][vehicles[j]].varValue)):
                bess_state[i].append("discharging")
                soc_bess[i].append(soc_bess[i][-1]-ex_charging_rate_bess)
            for k in range(int(s[s_names[i]][vehicles[j]].varValue-b[s_names[i]][vehicles[j]].varValue)):
                if soc_bess[i][-1] >= bess_capacity:
                    bess_state[i].append("idle")
                    soc_bess[i].append(soc_bess[i][-1])
                else:
                    bess_state[i].append("charging")
                    soc_bess[i].append(min(bess_capacity,soc_bess[i][-1]+ex_charging_rate_bess))
                    
for vehicle in vehicles:
    if len(state[vehicle]) < max_len:
        for i in range(max_len-len(state[vehicle])):
            state[vehicle].append("reached")
max_len_bess = 0
for i in range(len(s_names)):
    max_len_bess = max(max_len_bess,len(soc_bess[i]))
for i in range(len(soc_bess)):
    for j in range(max_len_bess-len(soc_bess[i])):
        bess_state[i].append("idle")
        soc_bess[i].append(soc_bess[i][-1])
for i in range(len(s_names)):
    soc_bess[i].pop(0)

wait_time = {}
for vehicle in  vehicles:
    wait_time[vehicle] = []
for vehicle in vehicles:
    for station in s_names:
        wait_time[vehicle].append((u[station][vehicle].varValue-a[station][vehicle].varValue)/time_discre)


import seaborn as sns
import matplotlib.pyplot as plt

plot = [[] for i in range(len(vehicles))]
for i in range(len(vehicles)):
    plot[i]=wait_time[vehicles[i]]

import numpy as np  
import matplotlib.pyplot as plt  
X =s_names[:]
plt.figure(figsize=(5,5))  
X_axis = np.arange(len(X)) 
for i in range(len(vehicles)):
    plt.bar(X_axis - 0.1*(i-(len(vehicles)//2)), plot[i], 0.1, label = vehicles[i])
plt.xticks(X_axis, X) 
plt.xlabel("station") 
plt.ylabel("Wait time") 
plt.title("Truck wait time") 
plt.legend() 
plt.show() 


plt.figure(figsize=(5,5))
plt.style.use('ggplot')
# create data 
x = list(range(len(state[vehicles[0]])))
for i in range(len(x)):
    x[i] = x[i]/time_discre 
# plot lines 
for vehicle in vehicles:
    plt.plot(x, state[vehicle], label = vehicle) 
plt.xlabel("Time") 
plt.ylabel("Truck state") 
plt.title("Truck schedule") 
plt.legend() 
plt.show() 

plt.figure(figsize=(5,5))

x = list(range(max_len_soc))
for i in range(len(x)):
    x[i] = x[i]/time_discre

# plot lines 
for vehicle in vehicles:
    plt.plot(x, soc_trucks[vehicle], label = vehicle)
plt.xlabel("Time") 
plt.ylabel("Truck State of Charge") 
plt.title("SoC of Trucks")  
plt.legend() 
plt.show()

plt.figure(figsize=(5,5))
x = list(range(len(bess_state[0]))) 
for i in range(len(x)):
    x[i] = x[i]/time_discre
# plot lines 
for i in range(len(s_names)):
    plt.plot(x, bess_state[i], label = f"Station{i+1}") 
plt.xlabel("Time") 
plt.ylabel("bess state") 
plt.title("bess schedule") 
plt.legend() 
plt.show() 

plt.figure(figsize=(5,5))

x = list(range(len(soc_bess[0])))
for i in range(len(x)):
    x[i] = x[i]/time_discre
# plot lines 
for i in range(len(s_names)):
    plt.plot(x, soc_bess[i], label = f"Station{i+1}") 
plt.xlabel("Time") 
plt.ylabel("BESS State of Charge") 
plt.title("SoC of BESS")  
plt.legend() 
plt.show()


wait and charging time is  -6000.0


TypeError: unsupported operand type(s) for /: 'tuple' and 'float'

In [8]:
pulp.LpStatus[cs.status]

'Optimal'